In [1]:
from IPython.display import display, HTML


display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob
from pyspark.sql.types import *

from pyspark.sql import SparkSession
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .getOrCreate()
sc = spark.sparkContext

23/10/18 17:22:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### vacinação

In [2]:
#df = spark.read.parquet("/dados10t/datalake/standard/dados_datasus/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/")
df = spark.read.parquet("/dados10t/datalake/standard/vigivac/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/")

In [3]:
df.count()

183759941

In [4]:
vac_long = df.selectExpr("id_vigvac","stack(5,'d1_data_aplicacao',d1_data_aplicacao,'d2_data_aplicacao',d2_data_aplicacao,'d3_data_aplicacao',d3_data_aplicacao,'d4_data_aplicacao',d4_data_aplicacao,'d5_data_aplicacao',d5_data_aplicacao) as (variable,data_entrada)" ).filter(F.col("data_entrada").isNotNull())

In [5]:
vac_long.count()

446217353

In [6]:
#vac_long_v2 = vac_long.rdd.zipWithIndex() 
#vac_long_v2 = vac_long_v2.toDF()
#vac_long_v2.printSchema()
#vac_long_v3=vac_long_v2.select(F.col("_1.*"),F.col("_2").alias('increasing_id'))
#vac_long_v3.printSchema()
#vac_long_v3.select("increasing_id").distinct().count()

In [7]:
vac_long.show(20,False)

+------------+-----------------+------------+
|id_vigvac   |variable         |data_entrada|
+------------+-----------------+------------+
|463856467968|d1_data_aplicacao|2021-05-28  |
|463856467968|d2_data_aplicacao|2021-08-26  |
|463856467969|d1_data_aplicacao|2021-09-11  |
|463856467970|d1_data_aplicacao|2021-06-23  |
|463856467970|d2_data_aplicacao|2021-10-19  |
|463856467970|d3_data_aplicacao|2022-03-02  |
|463856467971|d1_data_aplicacao|2021-09-16  |
|463856467971|d2_data_aplicacao|2021-11-12  |
|463856467972|d1_data_aplicacao|2021-06-18  |
|463856467972|d2_data_aplicacao|2021-09-10  |
|463856467973|d1_data_aplicacao|2021-10-20  |
|463856467973|d2_data_aplicacao|2022-01-03  |
|463856467974|d1_data_aplicacao|2021-07-07  |
|463856467974|d2_data_aplicacao|2022-02-19  |
|463856467975|d1_data_aplicacao|2021-07-22  |
|463856467975|d2_data_aplicacao|2021-10-14  |
|463856467976|d1_data_aplicacao|2021-05-31  |
|463856467976|d2_data_aplicacao|2021-07-26  |
|463856467977|d1_data_aplicacao|20

#### srag

In [8]:
srag = spark.read.parquet("/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")

In [9]:
srag_v2 = srag.select("id_vigvac","dt_notif_dt","dt_sin_pri_dt","dt_interna_dt","dt_encerra_dt","dt_saiduti_dt","evolucao","dt_evoluca","dt_pcr_dt","dt_entuti_dt")

In [10]:
srag_v2.count()

2487085

#### Evolução do caso
 - 1-Cura
 - 2-Óbito
 - 3- Óbito por outras causas
 - 9-Ignorado

In [11]:
srag_v2.groupBy("evolucao").count().orderBy(F.desc("count")).show(300,False) 

+--------+-------+
|evolucao|count  |
+--------+-------+
|1       |1531743|
|2       |593018 |
|null    |266170 |
|9       |61700  |
|3       |34454  |
+--------+-------+



In [12]:
srag_v2.filter(F.col("evolucao").isin([2,3]) ).count() #quantidade de óbitos no srag

627472

In [13]:
srag_v2.filter(F.col("dt_evoluca").isNotNull() ).count()

2066997

In [14]:
srag_v2 = srag_v2.withColumn("dt_evoluca_dt",F.to_date("dt_evoluca","dd/MM/yyyy"))

In [15]:
srag_v2.filter(F.col("dt_evoluca_dt").isNotNull() ).count()

2066997

In [16]:
srag_v2 = srag_v2.withColumn("dt_obito",F.when(F.col("evolucao").isin([2,3]),F.col("dt_evoluca_dt")   ).otherwise(None) )

In [17]:
srag_v2.count()

2487085

In [18]:
#srag.printSchema()

In [19]:
srag_v2 = srag_v2.withColumn("data_entrada", F.coalesce(F.col("dt_sin_pri_dt"), F.col("dt_notif_dt"), F.col("dt_interna_dt")  ) ) 
srag_v2 = srag_v2.withColumn("data_saida",F.coalesce(F.col("dt_obito"),F.col("dt_saiduti_dt"),F.col("dt_encerra_dt") ))   #,F.col("dt_encerra_dt"),F.col("dt_interna_dt"),F.col("dt_entuti_dt") ,F.col("dt_pcr_dt")  ) )  

In [20]:
print("N não nulos data entrada srag {}".format(srag_v2.filter(F.col("data_entrada").isNotNull() ).count()))
print("N não nulos data saida srag {}".format(srag_v2.filter(F.col("data_saida").isNotNull() ).count()) ) #somente com dt do óbito 625465/ dt do obito + dt_saiduti_dt 835587 / dt do obito + dt_saiduti_dt + dt_saiduti_dt 2244081

N não nulos data entrada srag 2487085
N não nulos data saida srag 2244081


In [21]:
2487085-2244081

243004

# 243004 sem data de saida no srag melhorar e 339 casos onde a data de entrada e maior que a de saida
##### casos por exemplo que a data da internação e anterior a data dos primeiros sintomas entre outros

In [22]:
srag_v2.filter(F.col("data_entrada") > F.col("data_saida") ).count()

339

In [23]:
srag_v2.filter(F.col("data_entrada") > F.col("data_saida") ).show(5)

+-------------+-----------+-------------+-------------+-------------+-------------+--------+----------+----------+------------+-------------+--------+------------+----------+
|    id_vigvac|dt_notif_dt|dt_sin_pri_dt|dt_interna_dt|dt_encerra_dt|dt_saiduti_dt|evolucao|dt_evoluca| dt_pcr_dt|dt_entuti_dt|dt_evoluca_dt|dt_obito|data_entrada|data_saida|
+-------------+-----------+-------------+-------------+-------------+-------------+--------+----------+----------+------------+-------------+--------+------------+----------+
|1511829181029| 2021-03-28|   2021-03-28|   2021-03-19|   2021-06-07|   2021-03-23|       1|09/04/2021|      null|  2021-03-19|   2021-04-09|    null|  2021-03-28|2021-03-23|
| 747324673800| 2021-11-04|   2021-10-31|   2021-09-13|   2021-12-16|   2021-10-06|       1|10/12/2021|2021-12-01|  2021-09-14|   2021-12-10|    null|  2021-10-31|2021-10-06|
| 412316987172| 2021-07-22|   2021-07-12|   2021-04-17|   2021-10-16|   2021-06-18|       1|24/07/2021|2021-07-22|  2021-06-0

In [24]:
#srag_v2.show(5) #.select("dt_encerra_dt","dt_saiduti_dt","dt_obito","data_saida")

In [25]:
total = srag_v2.count()

i = 0
for variavel in srag_v2.columns:
    i = i+1
    nao_vazios = srag_v2.filter(F.col(variavel).isNull()).count()
    porcent = nao_vazios * 100 / total
    print("% não nulos {} : {}".format(variavel,100 - porcent))
print(i)

% não nulos id_vigvac : 100.0
% não nulos dt_notif_dt : 100.0
% não nulos dt_sin_pri_dt : 100.0
% não nulos dt_interna_dt : 93.50758015910192
% não nulos dt_encerra_dt : 89.69994994139726
% não nulos dt_saiduti_dt : 17.371058890226905
% não nulos evolucao : 89.29791301865437
% não nulos dt_evoluca : 83.1092222421027
% não nulos dt_pcr_dt : 68.88896036926764
% não nulos dt_entuti_dt : 29.072307540755546
% não nulos dt_evoluca_dt : 83.1092222421027
% não nulos dt_obito : 25.148517240062162
% não nulos data_entrada : 100.0
% não nulos data_saida : 90.22936489906859
14


In [26]:
srag_datas = srag_v2.select("id_vigvac","data_entrada","data_saida" ).withColumn("variable",F.lit("srag"))#.withColumnRenamed("data_entrada","value")

In [27]:
periodo_obs = vac_long.unionByName(srag_datas,allowMissingColumns=True)


In [28]:
periodo_obs.show(20)

+------------+-----------------+------------+----------+
|   id_vigvac|         variable|data_entrada|data_saida|
+------------+-----------------+------------+----------+
|463856467968|d1_data_aplicacao|  2021-05-28|      null|
|463856467968|d2_data_aplicacao|  2021-08-26|      null|
|463856467969|d1_data_aplicacao|  2021-09-11|      null|
|463856467970|d1_data_aplicacao|  2021-06-23|      null|
|463856467970|d2_data_aplicacao|  2021-10-19|      null|
|463856467970|d3_data_aplicacao|  2022-03-02|      null|
|463856467971|d1_data_aplicacao|  2021-09-16|      null|
|463856467971|d2_data_aplicacao|  2021-11-12|      null|
|463856467972|d1_data_aplicacao|  2021-06-18|      null|
|463856467972|d2_data_aplicacao|  2021-09-10|      null|
|463856467973|d1_data_aplicacao|  2021-10-20|      null|
|463856467973|d2_data_aplicacao|  2022-01-03|      null|
|463856467974|d1_data_aplicacao|  2021-07-07|      null|
|463856467974|d2_data_aplicacao|  2022-02-19|      null|
|463856467975|d1_data_aplicacao

se o registro é diferente de um registro de srag, nesse ponto temos dados somente de vacina, então a data de saida fica sendo a data de entrada

In [29]:
periodo_obs = periodo_obs.withColumn("data_saida",F.when(F.col("variable") != "srag", F.col("data_entrada")).otherwise(F.col("data_saida") ) ) 

In [30]:
periodo_obs.show(10)

+------------+-----------------+------------+----------+
|   id_vigvac|         variable|data_entrada|data_saida|
+------------+-----------------+------------+----------+
|463856467968|d1_data_aplicacao|  2021-05-28|2021-05-28|
|463856467968|d2_data_aplicacao|  2021-08-26|2021-08-26|
|463856467969|d1_data_aplicacao|  2021-09-11|2021-09-11|
|463856467970|d1_data_aplicacao|  2021-06-23|2021-06-23|
|463856467970|d2_data_aplicacao|  2021-10-19|2021-10-19|
|463856467970|d3_data_aplicacao|  2022-03-02|2022-03-02|
|463856467971|d1_data_aplicacao|  2021-09-16|2021-09-16|
|463856467971|d2_data_aplicacao|  2021-11-12|2021-11-12|
|463856467972|d1_data_aplicacao|  2021-06-18|2021-06-18|
|463856467972|d2_data_aplicacao|  2021-09-10|2021-09-10|
+------------+-----------------+------------+----------+
only showing top 10 rows



#### sg

In [31]:
sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_09072022_v9.parquet/")

In [32]:
sg.count()

82896519

In [33]:
sg.printSchema()

root
 |-- dt_datanotificacao: date (nullable = true)
 |-- dt_datainiciosintomas: date (nullable = true)
 |-- dt_datateste: date (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- classificacaofinal: string (nullable = true)
 |-- dt_dataencerramento: date (nullable = true)
 |-- evolucaocaso: string (nullable = true)
 |-- tipoteste: string (nullable = true)
 |-- resultadoteste: string (nullable = true)
 |-- codigotipoteste: string (nullable = true)
 |-- codigoresultadoteste: string (nullable = true)
 |-- estadonotificacao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: double (nullable = true)
 |-- codigo_ibge_resid: string (nullable = true)
 |-- profissionalsaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- evolucaocaso_2: string (nullable = true)
 |-- classificacaofinal_2: string (nullable = true)
 |-- sexo_2: string (nullable = true)
 |-- racac

In [34]:
sg_v2 = sg.select("id_vigvac","dt_datanotificacao","dt_datainiciosintomas","dt_datateste","dt_dataencerramento","evolucaocaso_2")

In [35]:
total = sg_v2.count()

i = 0
for variavel in sg_v2.columns:
    i = i+1
    nao_vazios = sg_v2.filter(F.col(variavel).isNull()).count()
    porcent = nao_vazios * 100 / total
    print("% não nulos {} : {}".format(variavel,100 - porcent))
print(i)

% não nulos id_vigvac : 100.0


% não nulos dt_datanotificacao : 99.99959467538076


% não nulos dt_datainiciosintomas : 91.61799664953362


% não nulos dt_datateste : 83.51046562039596


% não nulos dt_dataencerramento : 42.60621486410063


% não nulos evolucaocaso_2 : 39.73388074353279
6


In [36]:
sg_datas = sg_v2.withColumn("data_entrada",F.coalesce(F.col("dt_datainiciosintomas") , F.col("dt_datanotificacao"), F.col("dt_datateste")  ) )
sg_datas = sg_datas.withColumn("data_saida",F.coalesce(F.col("dt_dataencerramento"),F.col("dt_datateste"),F.col("dt_datanotificacao") )  ).withColumn("variable",F.lit("sg"))#.select("id_vigvac","data_entrada","data_saida","variable")

In [37]:
sg_v2.count()

82896519

In [38]:
print("N não nulos data entrada sg {}".format(sg_datas.filter(F.col("data_entrada").isNotNull() ).count()))
print("N não nulos data saida sg {}".format(sg_datas.filter(F.col("data_saida").isNotNull() ).count()) )

N não nulos data entrada sg 82896502


N não nulos data saida sg 82896257


In [39]:
82896519-82896257

262

# 33701 sem data de saida no srag melhorar e 1292 casos onde a data de entrada e maior que a de saida
##### casos por exemplo que a data da internação e anterior a data dos primeiros sintomas entre outros

In [40]:
sg_datas.filter(F.col("data_entrada") > F.col("data_saida") ).count()

2844630

In [41]:
sg_datas.filter(F.col("data_entrada") > F.col("data_saida") ).show(5)

+------------+------------------+---------------------+------------+-------------------+--------------+------------+----------+--------+
|   id_vigvac|dt_datanotificacao|dt_datainiciosintomas|dt_datateste|dt_dataencerramento|evolucaocaso_2|data_entrada|data_saida|variable|
+------------+------------------+---------------------+------------+-------------------+--------------+------------+----------+--------+
|463856468113|        2020-06-27|                 null|  2020-06-17|               null|     Cancelado|  2020-06-27|2020-06-17|      sg|
|463856468133|        2022-01-08|                 null|  2022-01-07|               null|          null|  2022-01-08|2022-01-07|      sg|
|463856468213|        2022-02-23|                 null|  2022-01-24|               null|          null|  2022-02-23|2022-01-24|      sg|
|463856468354|        2022-01-28|                 null|  2022-01-11|               null|          null|  2022-01-28|2022-01-11|      sg|
|463856468395|        2021-03-31|        

In [42]:
sg_datas_v2 = sg_datas.select("id_vigvac","data_entrada","data_saida","evolucaocaso_2","variable")

### sg

In [43]:
sg_datas_v2.show(10)

+------------+------------+----------+--------------------+--------+
|   id_vigvac|data_entrada|data_saida|      evolucaocaso_2|variable|
+------------+------------+----------+--------------------+--------+
|463856467979|  2021-05-31|2021-05-31|                null|      sg|
|463856467983|  2021-03-22|2021-05-24|                Cura|      sg|
|463856467985|  2021-01-10|2022-01-13|                null|      sg|
|463856467990|  2022-01-23|2022-01-24|                null|      sg|
|463856467992|  2021-01-24|2021-02-02|Em tratamento dom...|      sg|
|463856467998|  2022-01-15|2022-01-19|                null|      sg|
|463856467999|  2021-02-14|2021-02-16|                null|      sg|
|463856467999|  2021-02-14|2021-02-16|           Cancelado|      sg|
|463856467999|  2022-03-06|2022-03-11|                null|      sg|
|463856467999|  2021-04-08|2021-04-12|                null|      sg|
+------------+------------+----------+--------------------+--------+
only showing top 10 rows



In [44]:
periodo_obs_long = periodo_obs.unionByName(sg_datas_v2,allowMissingColumns=True)

In [45]:
periodo_obs_long.groupBy("variable").count().orderBy(F.desc("count")).show(300,False)

+-----------------+---------+
|variable         |count    |
+-----------------+---------+
|d1_data_aplicacao|183759941|
|d2_data_aplicacao|159314222|
|d3_data_aplicacao|84018885 |
|sg               |82896519 |
|d4_data_aplicacao|18936809 |
|srag             |2487085  |
|d5_data_aplicacao|187496   |
+-----------------+---------+



In [46]:
total = periodo_obs_long.count()

i = 0
for variavel in periodo_obs_long.columns:
    i = i+1
    nao_vazios = periodo_obs_long.filter(F.col(variavel).isNull()).count()
    porcent = nao_vazios * 100 / total
    print("% não nulos {} : {}".format(variavel,100 - porcent))
print(i)

% não nulos id_vigvac : 100.0


% não nulos variable : 100.0


% não nulos data_entrada : 99.9999968021126


% não nulos data_saida : 99.9542389838098


% não nulos evolucaocaso_2 : 6.196001637371012
5


In [47]:
#periodo_obs_long.filter(F.col("variable") == "srag").show(10)

In [48]:
#periodo_obs_long.filter(F.col("variable") == "sg").show(10)

In [49]:
#periodo_obs_long.show(100)

In [50]:
#F.col()

#periodo_obs_long = periodo_obs_long.withColumn(F.when()  )

In [51]:
periodo_obs_long_v2 = periodo_obs_long.selectExpr("id_vigvac","variable","stack(2,'data_entrada',data_entrada,'data_saida',data_saida) as (variable2,data)" )

In [52]:
periodo_obs_long_v2.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- variable: string (nullable = true)
 |-- variable2: string (nullable = true)
 |-- data: date (nullable = true)



In [53]:
periodo_obs_long_v2.show(5)

+------------+-----------------+------------+----------+
|   id_vigvac|         variable|   variable2|      data|
+------------+-----------------+------------+----------+
|463856467968|d1_data_aplicacao|data_entrada|2021-05-28|
|463856467968|d1_data_aplicacao|  data_saida|2021-05-28|
|463856467968|d2_data_aplicacao|data_entrada|2021-08-26|
|463856467968|d2_data_aplicacao|  data_saida|2021-08-26|
|463856467969|d1_data_aplicacao|data_entrada|2021-09-11|
+------------+-----------------+------------+----------+
only showing top 5 rows



In [54]:
tb_periodo_obs = periodo_obs_long_v2.groupBy("id_vigvac").agg(F.min("data").alias("observation_period_start_date") ,F.max("data").alias("observation_period_end_date") ).cache()

In [55]:
tb_periodo_obs.count() #188537327

188537327

In [56]:
tb_periodo_obs.show(5)

+------------+-----------------------------+---------------------------+
|   id_vigvac|observation_period_start_date|observation_period_end_date|
+------------+-----------------------------+---------------------------+
|463856467978|                   2021-05-25|                 2021-07-01|
|463856468033|                   2021-08-13|                 2021-11-25|
|463856468157|                   2021-05-25|                 2022-04-29|
|463856468260|                   2021-08-10|                 2022-03-02|
|463856468291|                   2021-11-08|                 2021-12-20|
+------------+-----------------------------+---------------------------+
only showing top 5 rows



In [57]:
#srag_v2.filter(F.col("id_vigvac") == "1065152481134").take(2)

In [58]:
#periodo_obs_long_v2.filter(F.col("id_vigvac").isin(['1511828488212','1511828488233','1511828488237','1511828488669','1065152481134','463856469127'])).orderBy("id_vigvac").show(100)

In [59]:
#tb_periodo_obs.filter(F.col("id_vigvac").isin(['1511828488212','1511828488233','1511828488237','1511828488669','1065152481134','463856469127'])).orderBy("id_vigvac").show(100)

###  nessa versão 2 a data final é igual a data do update do banco de dados

In [60]:
tb_periodo_obs_opcao2 = tb_periodo_obs.withColumn("observation_period_end_date",F.to_date(F.lit("2022-07-30")))

#### a partir daqui vou tratar dois bancos , um com data final que é a data da extração e a data inicial é o primeiro evento, e um segundo banco onde a data final é a data do último evento registrado

In [61]:
tb_periodo_obs = tb_periodo_obs.withColumn("period_type_concept_id",F.lit("32817"))
tb_periodo_obs_opcao2 = tb_periodo_obs_opcao2.withColumn("period_type_concept_id",F.lit("32817"))

In [62]:
#tb_periodo_obs = tb_periodo_obs.withColumn("observation_period_id",F.monotonically_increasing_id())
tb_periodo_obs = tb_periodo_obs.withColumnRenamed("id_vigvac","person_id")

#tb_periodo_obs_opcao2 = tb_periodo_obs_opcao2.withColumn("observation_period_id",F.monotonically_increasing_id())
tb_periodo_obs_opcao2 = tb_periodo_obs_opcao2.withColumnRenamed("id_vigvac","person_id")

In [63]:
tb_periodo_obs.printSchema()

root
 |-- person_id: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = false)



In [64]:
tb_periodo_obs_opcao2.printSchema()

root
 |-- person_id: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = false)



In [65]:
tb_periodo_obs.filter(F.col("observation_period_start_date") > F.col("observation_period_end_date")).count()

0

In [66]:
tb_periodo_obs_opcao2.filter(F.col("observation_period_start_date") > F.col("observation_period_end_date")).count()

0

In [67]:
tb_periodo_obs.show(10)

+------------+-----------------------------+---------------------------+----------------------+
|   person_id|observation_period_start_date|observation_period_end_date|period_type_concept_id|
+------------+-----------------------------+---------------------------+----------------------+
|463856467978|                   2021-05-25|                 2021-07-01|                 32817|
|463856468033|                   2021-08-13|                 2021-11-25|                 32817|
|463856468157|                   2021-05-25|                 2022-04-29|                 32817|
|463856468260|                   2021-08-10|                 2022-03-02|                 32817|
|463856468291|                   2021-11-08|                 2021-12-20|                 32817|
|463856468413|                   2020-07-22|                 2022-02-04|                 32817|
|463856468511|                   2021-06-01|                 2021-06-28|                 32817|
|463856468748|                   2021-06

In [68]:
teste1 = tb_periodo_obs.withColumn("days",F.datediff("observation_period_end_date","observation_period_start_date"))

teste2 = tb_periodo_obs_opcao2.withColumn("days",F.datediff("observation_period_end_date","observation_period_start_date"))

In [69]:
teste1.describe("days").show()

+-------+------------------+
|summary|              days|
+-------+------------------+
|  count|         188537327|
|   mean|206.42901999984332|
| stddev|165.85111592540815|
|    min|                 0|
|    max|             44726|
+-------+------------------+



In [70]:
teste2.describe("days").show()

+-------+------------------+
|summary|              days|
+-------+------------------+
|  count|         188537327|
|   mean| 412.8456537362493|
| stddev|141.81791976279638|
|    min|                22|
|    max|             44772|
+-------+------------------+



In [71]:
teste1.filter(F.col("days") == 0).count()

20521728

In [72]:
teste2.filter(F.col("days") == 0).count()

0

In [73]:
tb_periodo_obs_opcao2.printSchema()

root
 |-- person_id: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = false)



In [74]:
tb_periodo_obs_opcao2 = tb_periodo_obs_opcao2.select("person_id","observation_period_start_date","observation_period_end_date","period_type_concept_id")

In [75]:
pacientes_v6 = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet/")

In [76]:
pacientes_v6.count() # 187634532

188537327

In [77]:
pacientes_v6.printSchema()

root
 |-- raca: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: string (nullable = true)
 |-- race_concept_id: string (nullable = true)
 |-- year_of_birth: long (nullable = true)
 |-- month_of_birth: long (nullable = true)
 |-- day_of_birth: long (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)



In [78]:
tb_periodo_obs_opcao2.printSchema()

root
 |-- person_id: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = false)



In [79]:
pacientes_join = pacientes_v6.select("person_id","id_vigvac")

In [80]:
tb_periodo_obs_opcao2 = tb_periodo_obs_opcao2.withColumnRenamed("person_id","id_vigvac")

In [81]:
tb_periodo_obs_opcao2_v2 = tb_periodo_obs_opcao2.join(pacientes_join,['id_vigvac'],'inner')

In [82]:
tb_periodo_obs_opcao2_v2.count()

188537327

In [83]:
tb_periodo_obs_opcao2_v2.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = false)
 |-- person_id: long (nullable = true)



In [84]:
import time
start = time.time()

In [85]:
tb_periodo_obs_opcao2_v3 = tb_periodo_obs_opcao2_v2.rdd.zipWithIndex().toDF().select(F.col("_1.*"),F.col("_2").alias('observation_period_id'))

In [86]:
end = time.time()
print(end - start)

70.29584455490112


In [87]:
tb_periodo_obs_opcao2_v3.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)
 |-- observation_period_id: long (nullable = true)



In [88]:
tb_periodo_obs_opcao2_v3.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_observation_period.parquet") #OBSERVATION_PERIOD

In [90]:
tb_periodo_obs_opcao2_v3.select("observation_period_id").distinct().count()

188537327

In [ ]:
188537327 == 

In [89]:
end = time.time()
print(end - start)

196.29272866249084
